# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import time

!pip install pandas_profiling==1.4.1 -q
import pandas_profiling

from matplotlib import pyplot
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, LassoCV
                            
from sklearn.ensemble import RandomForestRegressor

!pip install catboost
import catboost as ctb
from catboost import CatBoostRegressor

!pip install lightgbm
import lightgbm as lgbm
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error #MSE

from sklearn.preprocessing import OrdinalEncoder 
from sklearn.preprocessing import StandardScaler 


## Подготовка данных

In [2]:
try:
  # загрузка с сервера Яндекс
  df = pd.read_csv('/datasets/autos.csv')#, sep='\,')
  
except:
  #загрузка файла с гугл диска
  from google.colab import drive
  drive.mount('/content/drive')
  df = pd.read_csv('/content/drive/MyDrive/Data_CSV/autos.csv')#, sep='\t') 

In [3]:
df.head(3)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46


Для начала приведем в порядок названия стольбцов

In [4]:
# сделаем змеиный регистр
df.columns = df.columns.str.lower()
df.rename(columns = {'datecrawled':'date_crawled',
                    'vehicletype':'vehicle_type',
                    'registrationyear': 'registration_year',
                    'gearbox':'gear_box',
                    'registrationmonth':'registration_month',
                    'fueltype':'fuel_type',
                    'notrepaired':'not_repaired',                    
                    'datecreated':'date_created',
                    'numberofpictures':'number_of_pictures',
                    'postalcode':'postal_code',
                    'lastseen':'last_seen'},
            inplace = True)

### Далее ознакомимся с составом данных

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        354369 non-null  object
 1   price               354369 non-null  int64 
 2   vehicle_type        316879 non-null  object
 3   registration_year   354369 non-null  int64 
 4   gear_box            334536 non-null  object
 5   power               354369 non-null  int64 
 6   model               334664 non-null  object
 7   kilometer           354369 non-null  int64 
 8   registration_month  354369 non-null  int64 
 9   fuel_type           321474 non-null  object
 10  brand               354369 non-null  object
 11  not_repaired        283215 non-null  object
 12  date_created        354369 non-null  object
 13  number_of_pictures  354369 non-null  int64 
 14  postal_code         354369 non-null  int64 
 15  last_seen           354369 non-null  object
dtypes:

Немаленький датасет: 354 369 записей. Типы данных соответствуют содержанию, если не принимать во внимание признаки дат. При этом стоит проанализировать пригодность некоторых данных для целей проекта. Так, например, дата скачивания анкеты _("date_crawled")_ врядли имеет какое-либо влияние на цену продажи авто. Это же касается и признака дата создания анкеты _(" date_created ")_. 

Т.е. необходимо будет проверить влияние некоторых признаков на цену продажи с тем, чтобы скорректировать набор обучающих признаков.

In [6]:
round(df.isna().mean()*100,2).sort_values(ascending=False)

not_repaired          20.08
vehicle_type          10.58
fuel_type              9.28
gear_box               5.60
model                  5.56
date_crawled           0.00
price                  0.00
registration_year      0.00
power                  0.00
kilometer              0.00
registration_month     0.00
brand                  0.00
date_created           0.00
number_of_pictures     0.00
postal_code            0.00
last_seen              0.00
dtype: float64

In [7]:
df.duplicated().sum()

4

Качество данных не самое плохое: признаков с пропусками относительно не много. 

Всего 4 полных дубликата (и то не факт, может совпадение). 

Но признаки __"not_repaired"__ , __"vehicle_type"__  и __"fuel_type"__ содержат неприемлемое кол-во пропусков. Признаки __"gear_box"__ и __"model"__ также довольно разрежены. 

Далее необходимо будет оценить возможность/целесообразность заполнения пропусков, принимая во внимание, что некоторые алгоритмы, например CATBoost, нейтральны к пропускам.

Ниже рассмотрим каждый признак подробнее на предмет наличия/отсутсвия аномалий. Предварительно подготовим служебную функцию _def char(column)_для вывода характеристик столбца.

In [8]:
def char(column):
  print("\033[34m {}".format(''))
  print('Признак: "' + df.columns[column] + '"')
  print("\033[31m {}".format(''))
  print('Пропусков ', round(df[df.columns[column]].isna().mean()*100,2), ' %')
  print('---------------------------------------------------------------')

  #вывод характеристик цифровых признаков
  if df[df.columns[column]].dtypes=='int64':
    print("\033[30m {}".format(''))
    print(df[df.columns[column]].describe())
    df[df.columns[column]].plot(kind='hist', rot=45, figsize=(10,6), grid=True)
    plt.title('Распределение значений признака "' + df.columns[column] +'"')
    plt.ylabel("Количество значений")
    plt.show()
  
  #вывод характеристик категориальных признаков
  elif df[df.columns[column]].dtypes=='object':
    print("\033[30m {}".format(''))
    print('Уникальных значений признака: ',pd.Series(df[df.columns[column]].unique()).count())
    print('---------------------------------------------------------------')
    print(df[df.columns[column]].unique())
    print('---------------------------------------------------------------')
    print(df[df.columns[column]].value_counts())

  #return print('Пропусков ', round(df[df.columns[column]].isna().mean()*100,2), ' %')

In [9]:
#pandas_profiling.ProfileReport(df)

### Теперь приступим к изучению признаков и их влиянию на целевой признак "цена авто"

Начнем с "беспроблемных" признаков и далее перейдем к признакам с пропусками.

#### 0 "date_crawled"

In [10]:
char(0)

 
Признак: "date_crawled"
 
Пропусков  0.0  %
---------------------------------------------------------------
 
Уникальных значений признака:  271174
---------------------------------------------------------------
['2016-03-24 11:52:17' '2016-03-24 10:58:45' '2016-03-14 12:52:21' ...
 '2016-03-21 09:50:58' '2016-03-14 17:48:27' '2016-03-19 18:57:12']
---------------------------------------------------------------
2016-03-24 14:49:47    7
2016-03-19 21:49:56    6
2016-03-26 22:57:31    6
2016-03-07 17:36:19    5
2016-04-04 22:38:11    5
                      ..
2016-03-15 20:53:39    1
2016-03-29 18:46:39    1
2016-03-07 13:52:57    1
2016-03-14 19:36:42    1
2016-03-09 17:52:00    1
Name: date_crawled, Length: 271174, dtype: int64


Как видим, за один временной промежуток размером "1 минута" не более 7-ми скачиваний. Исходя из цели проекта, нам необходимо оценить влияние признака на стоимость авто.  Сделать это на основании такого представления данных не получиться. Требуется отформатировать даты.

In [11]:
df['date_crawled_new']=pd.to_datetime(df['date_crawled'], format='%Y-%m-%dT%H:%M:%S')
#df['date_crawled'].transform(pd.to_datetime(df['date_crawled'], format='%Y-%m-%dT%H:%M:%S'))
df.head(3)

,date_crawled,price,vehicle_type,registration_year,gear_box,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen,date_crawled_new
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57,2016-03-24 11:52:17
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50,2016-03-24 10:58:45
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46,2016-03-14 12:52:21


Далее посмотрим диапазон лет, месяцев, дней и т.д.

In [12]:
#диапазон лет 
print(pd.DatetimeIndex(df['date_crawled_new']).year.min())
print(pd.DatetimeIndex(df['date_crawled_new']).year.max())


2016
2016


In [13]:
#диапазон месяцев 
print(pd.DatetimeIndex(df['date_crawled_new']).month.min())
print(pd.DatetimeIndex(df['date_crawled_new']).month.max())

3
4


In [14]:
# диапазон дней
df['date_crawled_new'].max() - df['date_crawled_new'].min()

Timedelta('33 days 00:30:36')

In [15]:
#диапазон дат
print(df['date_crawled_new'].max())
print(df['date_crawled_new'].min())

2016-04-07 14:36:58
2016-03-05 14:06:22


Т.е. скачивания датируются мартом-апрелем 2016 г. и укладываются во временной диапазон 33 дня! Очень странный признак с точки зрения влияния на цену авто. 

Для верности посмотрим есть ли хоть какая-то зависимость цены авто от месяца скачивания анкеты.

In [16]:
df1 = pd.DataFrame(columns=['price','month'])
df1['price'] = df['price']
df1['month'] = pd.DatetimeIndex(df['date_crawled_new']).month
#df1.head()
pd.pivot_table(df1, values='price', index='month', columns=None, aggfunc=['count','mean','median'])#, sort=True)

,count,mean,median
,price,price,price
month,,,
3,296824,4383.801040,2700
4,57545,4586.130593,2899


Понятнее не стало. Принимая во внимание, что признак "дата скачивания анкеты из базы" __('date_crawled')__ имеет невнятное значение (кто скачивал? из какой базы? какую анкету? и т.п.) и невнятную временную фрагментацию, целесообразно из обучающего набора данных этот признак исключить. Соответсвенно, и искусственно добавленный столбец __'date_crawled_new'__ также следует удалить.

Продолжим

#### 1 "price"

In [17]:
char(1)

 
Признак: "price"
 
Пропусков  0.0  %
---------------------------------------------------------------
 
count    354369.000000
mean       4416.656776
std        4514.158514
min           0.000000
25%        1050.000000
50%        2700.000000
75%        6400.000000
max       20000.000000
Name: price, dtype: float64


Странные авто за 0 евро! Может "продавцы" перепутали "отдам в хорошие руки" и "продажа авто"?! Но аномальных цен нет. Продолжим.

In [18]:
df.query('price<=10')['price'].count()

12112

#### 3 "registration_year"

In [19]:
char(3)

 
Признак: "registration_year"
 
Пропусков  0.0  %
---------------------------------------------------------------
 
count    354369.000000
mean       2004.234448
std          90.227958
min        1000.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        9999.000000
Name: registration_year, dtype: float64


Авто из будущего!!!  А еще авто из раннего средневековья!!! До Леонардо!!!Интересено, сколько за них просят?

Не будем фантазерами: машины выпуска ранее 1950 г. это или хлам или раритет. На них ценообразование особенное. А машины после 2022 г. еще не выпустили. Так и отфильтруем.

In [20]:
# авто из будущего
df[df['registration_year']>2022]#['gear_box'].count()

,date_crawled,price,vehicle_type,registration_year,gear_box,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen,date_crawled_new
12946,2016-03-29 18:39:40,49,NaN,5000,NaN,0,golf,5000,12,NaN,volkswagen,NaN,2016-03-29 00:00:00,0,74523,2016-04-06 04:16:14,2016-03-29 18:39:40
15147,2016-03-14 00:52:02,0,NaN,9999,NaN,0,NaN,10000,0,NaN,sonstige_autos,NaN,2016-03-13 00:00:00,0,32689,2016-03-21 23:46:46,2016-03-14 00:52:02
15870,2016-04-02 11:55:48,1700,NaN,3200,NaN,0,NaN,5000,0,NaN,sonstige_autos,NaN,2016-04-02 00:00:00,0,33649,2016-04-06 09:46:13,2016-04-02 11:55:48
17271,2016-03-23 16:43:29,700,NaN,9999,NaN,0,other,10000,0,NaN,opel,NaN,2016-03-23 00:00:00,0,21769,2016-04-05 20:16:15,2016-03-23 16:43:29
17346,2016-03-06 16:06:20,6500,NaN,8888,NaN,0,NaN,10000,0,NaN,sonstige_autos,NaN,2016-03-06 00:00:00,0,55262,2016-03-30 20:46:55,2016-03-06 16:06:20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334967,2016-03-20 17:53:51,12000,NaN,4000,NaN,500,golf,5000,0,NaN,volkswagen,no,2016-03-20 00:00:00,0,57392,2016-04-07 00:46:30,2016-03-20 17:53:51
335727,2016-03-09 07:01:27,0,NaN,7500,manual,0,other,10000,0,petrol,mini,no,2016-03-09 00:00:00,0,9669,2016-03-19 19:44:50,2016-03-09 07:01:27
338829,2016-03-24 19:49:36,50,NaN,3000,NaN,3000,golf,100000,6,NaN,volkswagen,yes,2016-03-24 00:00:00,0,23992,2016-04-03 13:17:57,2016-03-24 19:49:36
340548,2016-04-02 17:44:03,0,NaN,3500,manual,75,NaN,5000,3,petrol,sonstige_autos,NaN,2016-04-02 00:00:00,0,96465,2016-04-04 15:17:51,2016-04-02 17:44:03


Их аж 105 штук!!! Во многих не указана модель. Цены странные. Пробег одинаковый. Кузов и коробка передач не указаны. __Похоже на "мусор". Удалим!__

In [21]:
df.drop(index=df[df['registration_year']>2022].index, inplace=True)

In [22]:
# авто из прошлого
df[df['registration_year']<1950]#['registration_year'].count()

,date_crawled,price,vehicle_type,registration_year,gear_box,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen,date_crawled_new
15,2016-03-11 21:39:15,450,small,1910,NaN,0,ka,5000,0,petrol,ford,NaN,2016-03-11 00:00:00,0,24148,2016-03-19 08:46:47,2016-03-11 21:39:15
622,2016-03-16 16:55:09,0,NaN,1111,NaN,0,NaN,5000,0,NaN,opel,NaN,2016-03-16 00:00:00,0,44628,2016-03-20 16:44:37,2016-03-16 16:55:09
1928,2016-03-25 15:58:21,7000,suv,1945,manual,48,other,150000,2,petrol,volkswagen,no,2016-03-25 00:00:00,0,58135,2016-03-25 15:58:21,2016-03-25 15:58:21
2273,2016-03-15 21:44:32,1800,convertible,1925,NaN,0,NaN,5000,1,NaN,sonstige_autos,no,2016-03-15 00:00:00,0,79288,2016-04-07 05:15:34,2016-03-15 21:44:32
6629,2016-04-02 13:47:16,0,small,1910,NaN,0,NaN,5000,1,other,sonstige_autos,NaN,2016-04-02 00:00:00,0,93105,2016-04-04 11:16:30,2016-04-02 13:47:16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346046,2016-03-26 16:45:28,18900,suv,1943,manual,60,other,150000,3,petrol,volkswagen,no,2016-03-26 00:00:00,0,51065,2016-03-26 16:45:28,2016-03-26 16:45:28
348830,2016-03-22 00:38:15,1,NaN,1000,NaN,1000,NaN,150000,0,NaN,sonstige_autos,NaN,2016-03-21 00:00:00,0,41472,2016-04-05 14:18:01,2016-03-22 00:38:15
351682,2016-03-12 00:57:39,11500,NaN,1800,NaN,16,other,5000,6,petrol,fiat,NaN,2016-03-11 00:00:00,0,16515,2016-04-05 19:47:27,2016-03-12 00:57:39
353531,2016-03-16 21:56:55,6000,sedan,1937,manual,38,other,5000,0,petrol,mercedes_benz,NaN,2016-03-16 00:00:00,0,23936,2016-03-30 18:47:41,2016-03-16 21:56:55


А этих даже больше, чем из будущего: 246! Не ну я знал, что Ford старая фирма. Но что на столько старая! С 1000 г. н.э. мондэо делает! И пробег всего 5000 км. __В мусорку!!!Всех!!!__

In [23]:
df.drop(index=df[df['registration_year']<1950].index, inplace=True)
df = df.reset_index(drop=True)

Проверим, что получилось после зачистки.

In [24]:
char(3)

 
Признак: "registration_year"
 
Пропусков  0.0  %
---------------------------------------------------------------
 
count    354018.000000
mean       2003.126301
std           7.303350
min        1950.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        2019.000000
Name: registration_year, dtype: float64


Ну теперь более вменяемо выглядит. Продолжим.



#### 5 "power"

In [25]:
char(5)

 
Признак: "power"
 
Пропусков  0.0  %
---------------------------------------------------------------
 
count    354018.000000
mean        110.119005
std         189.557911
min           0.000000
25%          69.000000
50%         105.000000
75%         143.000000
max       20000.000000
Name: power, dtype: float64


Авто без мотора и авто 20000 л.с.!!!Интересно. Опять же будем реалистами. Нас не интересуют авто без мотора (менее 20 л.с.) и мощностью более 500 л.с. Это всё про уникальные авто. Проверим.

In [26]:
# авто супер повер
df[df['power']>500]#['gear_box'].count()

,date_crawled,price,vehicle_type,registration_year,gear_box,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen,date_crawled_new
1814,2016-03-22 20:52:00,3200,small,2004,manual,1398,corolla,5000,6,petrol,toyota,no,2016-03-22 00:00:00,0,22043,2016-03-22 21:43:26,2016-03-22 20:52:00
2099,2016-03-21 11:55:22,0,sedan,1999,NaN,1799,vectra,150000,1,petrol,opel,yes,2016-03-21 00:00:00,0,1723,2016-04-04 04:49:06,2016-03-21 11:55:22
3742,2016-03-21 14:48:31,0,NaN,2017,manual,750,NaN,150000,8,petrol,smart,no,2016-03-21 00:00:00,0,49356,2016-03-24 03:44:59,2016-03-21 14:48:31
4056,2016-04-03 20:31:00,3100,sedan,2005,manual,953,colt,150000,4,gasoline,mitsubishi,no,2016-04-03 00:00:00,0,60326,2016-04-07 14:56:46,2016-04-03 20:31:00
5324,2016-03-29 19:44:48,500,wagon,1999,manual,1001,astra,150000,7,petrol,opel,NaN,2016-03-29 00:00:00,0,33154,2016-04-06 05:44:36,2016-03-29 19:44:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348620,2016-04-04 18:53:25,250,small,1999,manual,1241,ypsilon,150000,5,petrol,lancia,yes,2016-04-04 00:00:00,0,28259,2016-04-04 18:53:25,2016-04-04 18:53:25
351598,2016-03-07 21:36:19,1500,bus,2001,manual,1001,zafira,5000,7,gasoline,opel,no,2016-03-07 00:00:00,0,66117,2016-03-09 12:47:08,2016-03-07 21:36:19
353144,2016-04-02 20:54:21,12500,NaN,2017,manual,2000,other,60000,0,gasoline,chrysler,no,2016-04-02 00:00:00,0,44145,2016-04-06 21:44:39,2016-04-02 20:54:21
353283,2016-03-23 23:55:21,2400,sedan,2007,manual,650,c2,150000,8,petrol,citroen,NaN,2016-03-23 00:00:00,0,45277,2016-03-27 01:15:17,2016-03-23 23:55:21


Очень похоже, что народ путает мощность (л.с.) и объем двигателя (куб.см.). Можно заморочиться и попробовать восстановить, но у нас итак более 300 тыс. авто. 445 штук роли не играют. __Удаляем!__

In [27]:
df.drop(index=df[df['power']>500].index, inplace=True)
df = df.reset_index(drop=True)

In [28]:
# маломощные авто
df[df['power']<=20]#['power'].mean()

,date_crawled,price,vehicle_type,registration_year,gear_box,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen,date_crawled_new
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57,2016-03-24 11:52:17
31,2016-03-15 20:59:01,245,sedan,1994,NaN,0,golf,150000,2,petrol,volkswagen,no,2016-03-15 00:00:00,0,44145,2016-03-17 18:17:43,2016-03-15 20:59:01
36,2016-03-28 17:50:15,1500,NaN,2016,NaN,0,kangoo,150000,1,gasoline,renault,no,2016-03-28 00:00:00,0,46483,2016-03-30 09:18:02,2016-03-28 17:50:15
39,2016-03-26 22:06:17,0,NaN,1990,NaN,0,corsa,150000,1,petrol,opel,NaN,2016-03-26 00:00:00,0,56412,2016-03-27 17:43:34,2016-03-26 22:06:17
53,2016-03-17 07:56:40,4700,wagon,2005,manual,0,signum,150000,0,NaN,opel,no,2016-03-17 00:00:00,0,88433,2016-04-04 04:17:32,2016-03-17 07:56:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353550,2016-03-07 17:06:35,2600,NaN,2005,auto,0,c_klasse,150000,9,NaN,mercedes_benz,NaN,2016-03-07 00:00:00,0,61169,2016-03-08 21:28:38,2016-03-07 17:06:35
353564,2016-04-02 20:37:03,3999,wagon,2005,manual,3,3er,150000,5,gasoline,bmw,no,2016-04-02 00:00:00,0,81825,2016-04-06 20:47:12,2016-04-02 20:37:03
353567,2016-03-27 20:36:20,1150,bus,2000,manual,0,zafira,150000,3,petrol,opel,no,2016-03-27 00:00:00,0,26624,2016-03-29 10:17:23,2016-03-27 20:36:20
353568,2016-03-21 09:50:58,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21 00:00:00,0,2694,2016-03-21 10:42:49,2016-03-21 09:50:58


Ну тут всё гораздо хуже. Похоже что 40 тыс. "продавцов" решили не указывать мощность двигателя. Написали "0" и пр. "белиберду". Видимо решили, что итак данных хватает. Им то да. А нам то нет. С точки зрения обучения модели - это конечно не пропуски. Но очевидно же, что более 10% признака содержат "мусор". Попробуем обучить как есть. Но, если качество будет совсем уж неприемлемое, придется восстанавливать данные. __Удалять нельзя!__  

Продолжим.


#### 7 "kilometer"

In [29]:
char(7)

 
Признак: "kilometer"
 
Пропусков  0.0  %
---------------------------------------------------------------
 
count    353573.000000
mean     128310.023673
std       37759.637025
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: kilometer, dtype: float64


Здесь поводов для подозрений нет. Хотя подавляющее большинство машин с "хорошим таким" пробегом.

Двигаемся дальше.

#### 8 "registration_month"

In [30]:
char(8)

 
Признак: "registration_month"
 
Пропусков  0.0  %
---------------------------------------------------------------
 
count    353573.000000
mean          5.718856
std           3.724559
min           0.000000
25%           3.000000
50%           6.000000
75%           9.000000
max          12.000000
Name: registration_month, dtype: float64


Около 30 тыс. "продавцев" живет по своему календарю, где есть "0-й" месяц. Проверим.

In [31]:
# маломощные авто
df[df['registration_month']==0]#['power'].mean()

,date_crawled,price,vehicle_type,registration_year,gear_box,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen,date_crawled_new
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57,2016-03-24 11:52:17
9,2016-03-17 10:53:50,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06,2016-03-17 10:53:50
15,2016-04-01 12:46:46,300,NaN,2016,NaN,60,polo,150000,0,petrol,volkswagen,NaN,2016-04-01 00:00:00,0,38871,2016-04-01 12:46:46,2016-04-01 12:46:46
35,2016-03-11 11:50:37,1600,other,1991,manual,75,kadett,70000,0,NaN,opel,NaN,2016-03-11 00:00:00,0,2943,2016-04-07 03:46:09,2016-03-11 11:50:37
53,2016-03-17 07:56:40,4700,wagon,2005,manual,0,signum,150000,0,NaN,opel,no,2016-03-17 00:00:00,0,88433,2016-04-04 04:17:32,2016-03-17 07:56:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353509,2016-04-03 13:46:24,3500,NaN,1995,NaN,0,polo,150000,0,NaN,volkswagen,NaN,2016-04-03 00:00:00,0,74579,2016-04-05 12:44:38,2016-04-03 13:46:24
353522,2016-03-15 19:57:11,400,wagon,1991,manual,0,legacy,150000,0,petrol,subaru,NaN,2016-03-15 00:00:00,0,24558,2016-03-19 15:49:00,2016-03-15 19:57:11
353530,2016-03-31 19:36:18,1300,small,1999,manual,75,2_reihe,125000,0,NaN,peugeot,NaN,2016-03-31 00:00:00,0,35102,2016-04-06 13:44:44,2016-03-31 19:36:18
353533,2016-03-30 20:55:30,350,small,1996,NaN,65,punto,150000,0,NaN,fiat,NaN,2016-03-30 00:00:00,0,25436,2016-04-07 13:50:41,2016-03-30 20:55:30


Ну да. Таких 37 тыс. Среди них и те, кто мощность двигателя указывать не любит. 

Удалять нельзя! Восстанавливать не понятно как. 

Тогда вопрос: а так уж важен месяц регистрации, если год регистрации, например, 1996-й? Да и вообще. О какой регистрации идет речь? 

Понятно, что месяц - это скорее категориальный признак. Но вдруг...

In [32]:
df['price'].corr(df['registration_month'])

0.1103007457669738

Ну не то чтобы совсем никакой корреляции. Но 0.11 - так себе корреляция, слабенькая. А как насчет года регистрации.

In [33]:
df['price'].corr(df['registration_year'])

0.3798065737690396

Вот тут уже очевидно сильнее. При таком раскладе есть основания признак "месяц регистрации" (__"registration_month"__)из рабочего датасета удалить. 

Удаление всех избыточных признаков сделаем в конце раздела. А сейчас продолжим исследование признаков.

#### 10 "brand"

In [34]:
char(10)

 
Признак: "brand"
 
Пропусков  0.0  %
---------------------------------------------------------------
 
Уникальных значений признака:  40
---------------------------------------------------------------
['volkswagen' 'audi' 'jeep' 'skoda' 'bmw' 'peugeot' 'ford' 'mazda'
 'nissan' 'renault' 'mercedes_benz' 'opel' 'seat' 'citroen' 'honda' 'fiat'
 'mini' 'smart' 'hyundai' 'sonstige_autos' 'alfa_romeo' 'subaru' 'volvo'
 'mitsubishi' 'kia' 'suzuki' 'lancia' 'toyota' 'chevrolet' 'dacia'
 'daihatsu' 'trabant' 'saab' 'chrysler' 'jaguar' 'daewoo' 'porsche'
 'rover' 'land_rover' 'lada']
---------------------------------------------------------------
volkswagen        76875
opel              39837
bmw               36847
mercedes_benz     31984
audi              29405
ford              25126
renault           17897
peugeot           10985
fiat               9618
seat               6896
mazda              5608
skoda              5495
smart              5239
citroen            5125
nissan           

Ну тут хоть всё прилично. Продолжим.

#### 12 "date_created"

In [35]:
char(12)

 
Признак: "date_created"
 
Пропусков  0.0  %
---------------------------------------------------------------
 
Уникальных значений признака:  109
---------------------------------------------------------------
['2016-03-24 00:00:00' '2016-03-14 00:00:00' '2016-03-17 00:00:00'
 '2016-03-31 00:00:00' '2016-04-04 00:00:00' '2016-04-01 00:00:00'
 '2016-03-21 00:00:00' '2016-03-26 00:00:00' '2016-04-07 00:00:00'
 '2016-03-15 00:00:00' '2016-03-20 00:00:00' '2016-03-23 00:00:00'
 '2016-03-27 00:00:00' '2016-03-12 00:00:00' '2016-03-13 00:00:00'
 '2016-03-18 00:00:00' '2016-03-10 00:00:00' '2016-03-07 00:00:00'
 '2016-03-09 00:00:00' '2016-03-08 00:00:00' '2016-04-03 00:00:00'
 '2016-03-29 00:00:00' '2016-03-25 00:00:00' '2016-03-11 00:00:00'
 '2016-03-28 00:00:00' '2016-03-30 00:00:00' '2016-03-22 00:00:00'
 '2016-02-09 00:00:00' '2016-03-05 00:00:00' '2016-04-02 00:00:00'
 '2016-03-16 00:00:00' '2016-03-19 00:00:00' '2016-04-05 00:00:00'
 '2016-03-06 00:00:00' '2016-02-12 00:00:00' '2016-0

Видимо, следует повторить операции по аналогии с __"date_crawled"__

In [36]:
df['date_created_new']=pd.to_datetime(df['date_created'], format='%Y-%m-%dT%H:%M:%S')
#df['date_crawled'].transform(pd.to_datetime(df['date_crawled'], format='%Y-%m-%dT%H:%M:%S'))
df.head(3)

,date_crawled,price,vehicle_type,registration_year,gear_box,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen,date_crawled_new,date_created_new
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57,2016-03-24 11:52:17,2016-03-24
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50,2016-03-24 10:58:45,2016-03-24
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46,2016-03-14 12:52:21,2016-03-14


Далее посмотрим диапазон лет, месяцев, дней и т.д.

In [37]:
#диапазон лет 
print(pd.DatetimeIndex(df['date_created_new']).year.min())
print(pd.DatetimeIndex(df['date_created_new']).year.max())


2014
2016


In [38]:
#диапазон месяцев 
print(pd.DatetimeIndex(df['date_created_new']).month.min())
print(pd.DatetimeIndex(df['date_created_new']).month.max())

1
12


In [39]:
# диапазон дней
df['date_created_new'].max() - df['date_created_new'].min()

Timedelta('759 days 00:00:00')

In [40]:
#диапазон дат
print(df['date_created_new'].max())
print(df['date_created_new'].min())

2016-04-07 00:00:00
2014-03-10 00:00:00


Т.е. анкеты в исходном датасете были созданы в интервале между 10.03.2014 и 07.04.2016. Очень напоминает историю с датой скачивания.

Посмотрим есть ли зависимость цены авто от года/месяца создания анкеты.

In [41]:
df1 = pd.DataFrame(columns=['price','year','month'])
df1['price'] = df['price']
df1['year'] = pd.DatetimeIndex(df['date_created_new']).year
df1['month'] = pd.DatetimeIndex(df['date_created_new']).month
#df1.head()
pd.pivot_table(df1, values='price', index=['year','month'], columns=None, aggfunc=['count','mean','median'])#, sort=True)

count          mean   median
             price         price    price
year month                               
2014 3           1   8999.000000   8999.0
2015 3           1   6000.000000   6000.0
     6           1  10400.000000  10400.0
     8           2   9725.000000   9725.0
     9           3   3566.666667    600.0
     11          9   7753.888889   5999.0
     12          9   7249.888889   6950.0
2016 1          70   7922.885714   7449.5
     2         396   6186.744949   5274.5
     3      296095   4382.783634   2700.0
     4       56986   4581.185484   2899.0

Подавляющее большинство анкет создано в марте-апреле 2016 г. В остальных периодах "крохи". 

Все симптомы "болезни" как у признака __'date_crawled'__. Т.о. целесообразно из обучающего набора данных признак __'date_created'__ исключить. Соответственно, и искусственно добавленный столбец __'date_created_new'__ также следует удалить.

#### 13 "number_of_pictures"

In [42]:
char(13)

 
Признак: "number_of_pictures"
 
Пропусков  0.0  %
---------------------------------------------------------------
 
count    353573.0
mean          0.0
std           0.0
min           0.0
25%           0.0
50%           0.0
75%           0.0
max           0.0
Name: number_of_pictures, dtype: float64


Прекрасный признак. __На помойку!__

#### 14 "postal_code"

In [43]:
char(14)

 
Признак: "postal_code"
 
Пропусков  0.0  %
---------------------------------------------------------------
 
count    353573.000000
mean      50519.027434
std       25782.491049
min        1067.000000
25%       30165.000000
50%       49419.000000
75%       71088.000000
max       99998.000000
Name: postal_code, dtype: float64


Несмотря на то, что признак в формате int, это скорее категориальные значения.
Пропусков нет. Для проверки на аномальность надо знать систему почтовой индексации, вероятно, в США. Но я ничего такого делать не планирую. Извините.

Вот посмотреть на разницу в ценах на сопоставимые машины в разных регионах "индексах" кратенько можно.

In [44]:
print(df.query('brand=="audi"').
      query('model=="a2"').
      query('registration_year == 2016').
      query('power==75').
      query('kilometer==150000').
      groupby(by=['brand','model','registration_year', 'power','kilometer','postal_code'])['price'].agg(['count','min','max','mean']))#.to_markdown())

                                                           count   min   max  \
brand model registration_year power kilometer postal_code                      
audi  a2    2016              75    150000    12247            1  4000  4000   
                                              38895            2  2750  2900   
                                              50170            1  2780  2780   
                                              51103            1  4250  4250   
                                              64287            1  4900  4900   
                                              81476            1  4100  4100   
                                              85072            1  5000  5000   
                                              97506            1  3000  3000   

                                                           mean  
brand model registration_year power kilometer postal_code        
audi  a2    2016              75    150000    12247        4000  
 

Если "на глаз", то цена на сопоставимые машины заметно, местами почти в 2 раза, отличается. 

На всякий случай корреляцию...

In [45]:
df['price'].corr(df['postal_code'])

0.07589626783024993

Ну в цифровом виде почтовый код влияет очень слабо. Хотя некоторое влияние всё же имеется. Поэтому признак в рабочем датасете сохраним. Но вероятно придеться из него сделать формат object, чтобы затем подвергнуть OrdinalEncoding.

#### 15 "last_seen"

In [46]:
char(15) 

 
Признак: "last_seen"
 
Пропусков  0.0  %
---------------------------------------------------------------
 
Уникальных значений признака:  178829
---------------------------------------------------------------
['2016-04-07 03:16:57' '2016-04-07 01:46:50' '2016-04-05 12:47:46' ...
 '2016-03-19 20:44:43' '2016-03-29 10:17:23' '2016-03-21 10:42:49']
---------------------------------------------------------------
2016-04-06 13:45:54    17
2016-04-07 09:45:10    16
2016-04-07 00:45:17    16
2016-04-06 01:15:23    16
2016-04-07 13:17:48    16
                       ..
2016-03-16 04:16:44     1
2016-03-24 17:25:30     1
2016-03-09 10:39:34     1
2016-03-07 19:47:32     1
2016-03-08 16:18:24     1
Name: last_seen, Length: 178829, dtype: int64


Снова здорова! Обрабатываем как __"date_crawled"__

In [47]:
df['last_seen_new']=pd.to_datetime(df['last_seen'], format='%Y-%m-%dT%H:%M:%S')
#df['date_crawled'].transform(pd.to_datetime(df['date_crawled'], format='%Y-%m-%dT%H:%M:%S'))
df.head(3)

,date_crawled,price,vehicle_type,registration_year,gear_box,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen,date_crawled_new,date_created_new,last_seen_new
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57,2016-03-24 11:52:17,2016-03-24,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50,2016-03-24 10:58:45,2016-03-24,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46,2016-03-14 12:52:21,2016-03-14,2016-04-05 12:47:46


Далее посмотрим диапазон лет, месяцев, дней и т.д.

In [48]:
#диапазон лет 
print(pd.DatetimeIndex(df['last_seen_new']).year.min())
print(pd.DatetimeIndex(df['last_seen_new']).year.max())


2016
2016


In [49]:
#диапазон месяцев 
print(pd.DatetimeIndex(df['last_seen_new']).month.min())
print(pd.DatetimeIndex(df['last_seen_new']).month.max())

3
4


In [50]:
# диапазон дней
df['last_seen_new'].max() - df['last_seen_new'].min()

Timedelta('33 days 00:43:43')

In [51]:
#диапазон дат
print(df['last_seen_new'].max())
print(df['last_seen_new'].min())

2016-04-07 14:58:51
2016-03-05 14:15:08


Т.е. последние активности датируются мартом-апрелем 2016 г. и укладываются во временной диапазон 33 дня! Знакомая история.

Для верности посмотрим есть ли хоть какая-то зависимость цены авто от месяца последней активности.

In [52]:
df1 = pd.DataFrame(columns=['price','month'])
df1['price'] = df['price']
df1['month'] = pd.DatetimeIndex(df['last_seen_new']).month
#df1.head()
pd.pivot_table(df1, values='price', index='month', columns=None, aggfunc=['count','mean','median'])#, sort=True)

,count,mean,median
,price,price,price
month,,,
3,153800,3648.677412,2199
4,199773,5009.747594,3300


Понятнее снова не стало. Все симптомы "болезни" снова как у признака __'date_crawled'__. Т.о. целесообразно из обучающего набора данных признак __'last_seen'__ исключить. Соответственно, и искусственно добавленный столбец __'last_seen_new'__ также следует удалить.

Продолжим

#### 11 "not_repaired"

In [53]:
char(11)

 
Признак: "not_repaired"
 
Пропусков  20.01  %
---------------------------------------------------------------
 
Уникальных значений признака:  2
---------------------------------------------------------------
[nan 'yes' 'no']
---------------------------------------------------------------
no     246888
yes     35952
Name: not_repaired, dtype: int64


Проблема одна: про 21% авто не известно была машина в ремонте или нет. 

Восстановить эти данные невозможно. Удалить 21% датасета - неприемлемо. 

Придется заменить "nan" значением "unknown" и в дальнейшем подвергнуть признак OrdinalEncoding, если понадобиться.

In [54]:
df['not_repaired'] = df['not_repaired'].fillna('unknown')

Продолжим.

#### 2 "vehicle_type"

In [55]:
char(2)

 
Признак: "vehicle_type"
 
Пропусков  10.52  %
---------------------------------------------------------------
 
Уникальных значений признака:  8
---------------------------------------------------------------
[nan 'coupe' 'suv' 'small' 'sedan' 'convertible' 'bus' 'wagon' 'other']
---------------------------------------------------------------
sedan          91310
small          79728
wagon          65086
bus            28744
convertible    20177
coupe          16129
suv            11966
other           3250
Name: vehicle_type, dtype: int64


С типом кузова всё относительно хорошо, не считая высокой доли пропусков. Очевидно, что тип кузова имеет важное значение для покупателя автомобиля. Т.е. влияет на цену авто. Поэтому следует попытаться восстановить пропуски. 

Первым, что приходит на ум - это восстановление пропусков по марке/модели авто, которые уже есть в исходном датасете. Попробуем.


In [56]:
# краткий обзор пропусков по моделям
df[df['vehicle_type'].isna()].groupby(by=['brand','model'])['brand'].count()

brand       model   
alfa_romeo  145         14
            147         80
            156         46
            159         12
            other       25
                        ..
volvo       v40         68
            v50         11
            v60          1
            v70         24
            xc_reihe     7
Name: brand, Length: 283, dtype: int64

In [57]:
# краткий обзор какие бывают кузова по моделям
df.groupby(by=['brand','model','vehicle_type'])['vehicle_type'].count()

brand       model     vehicle_type
alfa_romeo  145       coupe             2
                      other             1
                      sedan            13
                      small            19
            147       coupe            27
                                     ... 
volvo       v70       coupe             1
                      wagon           608
            xc_reihe  sedan             3
                      suv             216
                      wagon            41
Name: vehicle_type, Length: 1408, dtype: int64

Как видим, разные модели одного и того же бренда могут быть в разных "кузовах". Т.е. точно восстановить "тип кузова" по бренду и модели не получиться. 

Придется заменить "nan" значением "unknown" и в дальнейшем подвергнуть признак OrdinalEncoding, если понадобиться.

In [58]:
df['vehicle_type'] = df['vehicle_type'].fillna('unknown')

#### 9 "fuel_type"

In [59]:
char(9)

 
Признак: "fuel_type"
 
Пропусков  9.22  %
---------------------------------------------------------------
 
Уникальных значений признака:  7
---------------------------------------------------------------
['petrol' 'gasoline' nan 'lpg' 'other' 'hybrid' 'cng' 'electric']
---------------------------------------------------------------
petrol      215972
gasoline     98628
lpg           5304
cng            562
hybrid         232
other          199
electric        89
Name: fuel_type, dtype: int64


Восстанавливать затруднительно, да и нецелесообразно.
Придется заменить "nan" значением "unknown" и в дальнейшем подвергнуть признак OrdinalEncoding, если понадобиться.

In [60]:
df['fuel_type'] = df['fuel_type'].fillna('unknown')

#### 6 "model"

In [61]:
char(6)

 
Признак: "model"
 
Пропусков  5.51  %
---------------------------------------------------------------
 
Уникальных значений признака:  250
---------------------------------------------------------------
['golf' nan 'grand' 'fabia' '3er' '2_reihe' 'other' 'c_max' '3_reihe'
 'passat' 'navara' 'polo' 'twingo' 'a_klasse' 'scirocco' '5er' 'meriva'
 'arosa' 'c4' 'civic' 'transporter' 'punto' 'e_klasse' 'clio' 'kadett'
 'kangoo' 'corsa' 'one' 'fortwo' '1er' 'b_klasse' 'signum' 'astra' 'a8'
 'jetta' 'fiesta' 'c_klasse' 'micra' 'vito' 'sprinter' '156' 'escort'
 'forester' 'xc_reihe' 'scenic' 'a4' 'ka' 'a1' 'insignia' 'combo' 'focus'
 'tt' 'a6' 'jazz' 'omega' 'slk' '7er' '80' '147' '100' 'z_reihe'
 'sportage' 'sorento' 'v40' 'ibiza' 'mustang' 'eos' 'touran' 'getz' 'a3'
 'almera' 'megane' 'lupo' 'r19' 'zafira' 'caddy' 'mondeo' 'cordoba' 'colt'
 'impreza' 'vectra' 'berlingo' 'tiguan' 'i_reihe' 'espace' 'sharan'
 '6_reihe' 'panda' 'up' 'seicento' 'ceed' '5_reihe' 'yeti' 'octavia' 'mii'
 'rx_reihe

In [62]:
#df.duplicated(subset=['brand','model'])

In [63]:
#sorted(df[df['model'].isna()==False]['model'].unique())

Есть неявные дубликаты:'range_rover', и  'rangerover'. Их имеет смысл привести к одному виду:'range_rover'.



In [64]:
df['model'] = df['model'].replace('rangerover', 'range_rover')

In [65]:
# краткий обзор пропусков по брендам кузовам 
df[df['model'].isna()].groupby(by=['brand','vehicle_type'])['brand'].count()

brand       vehicle_type
alfa_romeo  coupe           16
            other            1
            sedan           40
            small            7
            unknown         55
                            ..
volvo       coupe            1
            sedan           24
            suv              1
            unknown         32
            wagon           51
Name: brand, Length: 294, dtype: int64

Для восстановления модели авто данных недостаточно. Удалять 5,5% данных нецелесообразно. 

Предлагается заменить "nan" на "unknown". Хуже не будет, т.к. перед обучением для линейных моделей всё-равно придется OrdinalEncoding делать. А бустингам всё равно: одним значением признака больше - одним меньше. 

In [66]:
df['model'] = df['model'].fillna('unknown')

Продолжим

#### 4 "gear_box"

In [67]:
char(4)

 
Признак: "gear_box"
 
Пропусков  5.54  %
---------------------------------------------------------------
 
Уникальных значений признака:  2
---------------------------------------------------------------
['manual' 'auto' nan]
---------------------------------------------------------------
manual    267845
auto       66152
Name: gear_box, dtype: int64


Восстанавливать затруднительно, да и нецелесообразно.
Придется заменить "nan" значением "unknown" и в дальнейшем подвергнуть признак OrdinalEncoding, если понадобиться.

In [68]:
df['gear_box'] = df['gear_box'].fillna('unknown')

### Теперь исключим из рабочего набора данных признаки, которые выше были определены как "бесполезные".

In [69]:
#df = df.drop(['date_crawled','date_crawled_new',
              #'registration_month',
              #'date_created', 'date_created_new',
              #'last_seen','last_seen_new',
              #'number_of_pictures'], axis=1)


df = df[['price', 'vehicle_type', 'registration_year', 'gear_box', 'power',
       'model', 'kilometer', 'fuel_type', 'brand', 'not_repaired',
       'postal_code']]

Проверим результаты предобработки данных

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353573 entries, 0 to 353572
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   price              353573 non-null  int64 
 1   vehicle_type       353573 non-null  object
 2   registration_year  353573 non-null  int64 
 3   gear_box           353573 non-null  object
 4   power              353573 non-null  int64 
 5   model              353573 non-null  object
 6   kilometer          353573 non-null  int64 
 7   fuel_type          353573 non-null  object
 8   brand              353573 non-null  object
 9   not_repaired       353573 non-null  object
 10  postal_code        353573 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 29.7+ MB


In [71]:
round(df.isna().mean()*100,2).sort_values(ascending=False)

price                0.0
vehicle_type         0.0
registration_year    0.0
gear_box             0.0
power                0.0
model                0.0
kilometer            0.0
fuel_type            0.0
brand                0.0
not_repaired         0.0
postal_code          0.0
dtype: float64

Готово. После предобработки в нашем распоряжении осталось более 350-ти тысяч записей. Переходим к обучению моделей.

## Обучение моделей

Обучим несколько моделей регрессии:
1. LinearRegression
2. Случайный лес 
3. Lasso
4. CatBoost
5. LightGBM

Все модели применим "сырые", т.е. с гиперпараметрами по умолчанию (без указания гиперпараметров).
Качество предсказаний оценим для всех моделей метрикой RMSE.
Также зафиксируем время обучения и предсказания.


Для удобства анализа подготовим шаблон сводной таблицы результатов, функцию замера времени и функцию моделирования(принимает модель и обучающие наборы, выдаёт RMSE и время исполнения).

In [72]:
# шаблон сводной таблицы результатов моделирования 
results = []
results=pd.DataFrame(columns=['REGRESSOR','RMSE','Fit Time, sec', 'Prediction Time, sec', 'Total time, sec.'])

In [73]:
# функция замера времени операций
def exec_time(start, end):
   diff_time = end - start
   return round(diff_time,2)

In [74]:
# обучение моделей и сохранение результато в сводной таблице result
def modeling(model, features_train,target_train, features_valid, target_valid):
  
  global results

  #начинаем отсчет времени обучения
  start = time.time()
  #учим модель
  model.fit(features_train, target_train)
  #заканчиваем отсчет времени
  end = time.time()
  #фиксируем время обучения модели
  fit_time =  exec_time(start,end) 

  #начинаем отсчет времени предсказания
  start = time.time()
  #получаем предсказания
  predicted_valid = model.predict(features_valid)
  #заканчиваем отсчет времени предсказания
  end = time.time()
  #фиксируем время предсказания
  pred_time =  exec_time(start,end) 
   
  #оцениваем rmse
  rmse = round((mean_squared_error(target_valid, predicted_valid)**0.5),4)
   
  # заполняем таблицу результатов
  row = [model, rmse, fit_time, pred_time, fit_time + pred_time ]
  results.loc[len(results)] = row 

Подготовим наборы данных для моделирования. Сначала проведем OrdinalEncoding, а затем сформируем train и valid наборы.

In [75]:
#OE
#encoder = OrdinalEncoder() 
#df =  pd.DataFrame(encoder.fit_transform(df), columns=df.columns)
#df.head()

In [76]:
# набор обучающих признаков
features = df.drop('price', axis=1)
features.shape

(353573, 10)

In [77]:
#OE
encoder = OrdinalEncoder() 
features =  pd.DataFrame(encoder.fit_transform(features), columns=features.columns)
features.shape

(353573, 10)

In [78]:
# целевой признак
target = df['price']
target.shape

(353573,)

In [79]:
#разбиение на обучающую и валидационную выборки
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, train_size= 0.75, random_state=12345)

In [80]:
print('Обучающие параметры:', features_train.shape)
print('Обучающий целевой параметр:', target_train.shape)
print('-------------------------------------')
print('Валидационные параметры:', features_valid.shape)
print('Валидационный целевой параметр:', target_valid.shape)
print('-------------------------------------')
#print('Тестовые параметры:', features_test.shape)
#print('Тестовый целевой параметр:', target_test.shape)

Обучающие параметры: (265179, 10)
Обучающий целевой параметр: (265179,)
-------------------------------------
Валидационные параметры: (88394, 10)
Валидационный целевой параметр: (88394,)
-------------------------------------


Далее обучим "сырые" модели и рассчитаем метрики качества

#### Линейная регрессия

In [81]:
%%time
lr=LinearRegression()
modeling(lr, features_train,target_train, features_valid, target_valid)

CPU times: user 168 ms, sys: 175 ms, total: 344 ms
Wall time: 298 ms


#### Случайный лес

In [82]:
%%time
rfr = RandomForestRegressor(random_state=12345)
modeling(rfr, features_train,target_train, features_valid, target_valid)

CPU times: user 2min 16s, sys: 1.06 s, total: 2min 17s
Wall time: 2min 18s


#### Lasso регрессия

In [83]:
%%time
lasso = LassoCV(random_state=12345)
modeling(lasso, features_train,target_train, features_valid, target_valid)

CPU times: user 3.75 s, sys: 3.98 s, total: 7.73 s
Wall time: 7.71 s


#### CATBoost регрессия

In [84]:
%%time
cbr = CatBoostRegressor(random_state=12345, verbose=100)
modeling(cbr, features_train,target_train, features_valid, target_valid)

Learning rate set to 0.098878
0:	learn: 4240.6935827	total: 105ms	remaining: 1m 44s
100:	learn: 1951.9616026	total: 4.37s	remaining: 38.9s
200:	learn: 1857.6608152	total: 8.7s	remaining: 34.6s
300:	learn: 1812.3307833	total: 13s	remaining: 30.2s
400:	learn: 1780.7570875	total: 17.3s	remaining: 25.9s
500:	learn: 1757.0539762	total: 21.6s	remaining: 21.6s
600:	learn: 1736.6764139	total: 26s	remaining: 17.3s
700:	learn: 1719.2758122	total: 30.3s	remaining: 12.9s
800:	learn: 1704.1315565	total: 34.4s	remaining: 8.54s
900:	learn: 1690.2114374	total: 38.5s	remaining: 4.23s
999:	learn: 1678.4619979	total: 42.6s	remaining: 0us
CPU times: user 42.8 s, sys: 226 ms, total: 43 s
Wall time: 44.7 s


#### LightGBM регрессия

In [85]:
%%time
lgbm = LGBMRegressor(random_state=12345)
modeling(lgbm, features_train,target_train, features_valid, target_valid)

CPU times: user 42.3 s, sys: 442 ms, total: 42.7 s
Wall time: 43.2 s


## Анализ моделей

In [86]:
#colab метод to_markdown() выполняет, а тренажер Yandex нет. поэтому такая конструкция
try:
    print(results.sort_values(by='RMSE', ascending=True).to_markdown())
except:
    display(results.sort_values(by='RMSE', ascending=True))

,REGRESSOR,RMSE,"Fit Time, sec","Prediction Time, sec","Total time, sec."
1,"(DecisionTreeRegressor(max_features='auto', ra...",1714.1514,132.10,5.98,138.08
3,<catboost.core.CatBoostRegressor object at 0x7...,1735.7185,44.54,0.11,44.65
4,LGBMRegressor(random_state=12345),1827.3259,42.35,0.89,43.24
0,LinearRegression(),3168.4710,0.28,0.01,0.29
2,LassoCV(random_state=12345),3286.6591,7.64,0.06,7.70


По качеству предсказания самый высокий результат из "сырых" моделей выдал Случайный лес. Рейтинг по скорости обучения 4-й.    

CATBoost выдал второй по качеству результат, но при этом самое худшее время.

Заданный проектом LightGBM выдал третий по качеству результат за очень приличное (малое) время. При этом качество по отношению к Случайному лесу и CATBoost хуже не критично.

"Обычные" регрессоры Linear и Lasso сработали "почти мгновенно", но качество в 2 раза хуже!

**Таким образом, LightGBM по соотношению "время-качество" явно лидирует.**

Поскольку заданием проекта предусмотрено обязательное применение LightGBM, то имеет смысл проверить даст ли настройка гиперпараметров увеличение качества, при некритичном увеличении скорости обучения.

In [87]:
%%time
lgbm = LGBMRegressor(random_state=12345, learning_rate = 0.07, max_depth = 6, n_estimators=500)
modeling(lgbm, features_train,target_train, features_valid, target_valid)

CPU times: user 2min 17s, sys: 1.16 s, total: 2min 18s
Wall time: 2min 20s


In [88]:
#colab метод to_markdown() выполняет, а тренажер Yandex нет. поэтому такая конструкция
try:
    print(results.sort_values(by='RMSE', ascending=True).to_markdown())
except:
    display(results.sort_values(by='RMSE', ascending=True))

,REGRESSOR,RMSE,"Fit Time, sec","Prediction Time, sec","Total time, sec."
1,"(DecisionTreeRegressor(max_features='auto', ra...",1714.1514,132.10,5.98,138.08
3,<catboost.core.CatBoostRegressor object at 0x7...,1735.7185,44.54,0.11,44.65
5,"LGBMRegressor(learning_rate=0.07, max_depth=6,...",1747.8015,136.49,3.72,140.21
4,LGBMRegressor(random_state=12345),1827.3259,42.35,0.89,43.24
0,LinearRegression(),3168.4710,0.28,0.01,0.29
2,LassoCV(random_state=12345),3286.6591,7.64,0.06,7.70


Настройка 4-х гиперпараметров LightGBM вывела его на первое место по качеству предсказания (RMSE снизился с 1827.3259 до 1747.8015), но время работы выросло в 4-е раза (выросло с 7.69 сек. до 33.50 сек.). Результат стал заметно лучше, чем у Случайного леса и CATBoost, но время работы  __в три раза больше Случайного леса__.  

Понятно, что сравнение не вполне корректное, т.к. не все модели подвергнуты донастройке гиперпараметров.

### Выводы:





- В соответсвии с заданием для оценки стоимости авто была обучена модель регрессии LightGBM. 
- Кроме того, для сравнения, были обучены еще несколько моделей (случайный лес, CATBoost, Lasso и линейная регрессия). 
- Изначально модели были обучены без настройки гиперпараметров. 
- __По соотношению "время - качество" модель LightGBM заняла первое место среди обученных моделей.__ 
- После небольшой ручной донастройки гиперпараметров LightGBM дала незначительный прирост качества при трехкратном увеличении времени обучения/прогнозирования.
- После донастройки гиперпараметров __по соотношению "время - качество" модель LightGBM также заняла первое место среди всех обученных моделей.__ 


In [91]:
print('Спасибо за внимание!')

Спасибо за внимание!
